In [1]:
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import rioxarray as rio
from sklearn import linear_model
import statsmodels.api as ssm #for detail description of linear coefficients, intercepts, deviations, and many more
import xarray as xr

In [2]:

#Read Water Quality Data
water_quality_fp="../data/epd_water_quality/prc/epd_water_quality_1986_2022.csv"
water_quality_df=pd.read_csv(water_quality_fp).dropna()
summer_mean=pd.read_csv("../data/epd_water_quality/prc/summer_mean_1986_2022.csv")

vars=['chla_surf', 'diss_o_surf', 'ph_surf',
       'salinity_surf', 'turbidity_surf', 'temp_surf', 'suspended_solids_surf',
       'nitrates_surf','chla_bott', 'diss_o_bott', 'ph_bott',
       'salinity_bott', 'turbidity_bott', 'temp_bott', 'suspended_solids_bott',
       'nitrates_bott']

# Read EPD Stations Metadata
epd_stations_fp="../data/epd_water_quality/prc/epd_stations.csv"
epd_stations=pd.read_csv(epd_stations_fp)

## Linear Regression

##### Check for Colliniearty with VIF

In [3]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

#find design matrix for regression model using 'rating' as response variable 
y, X = dmatrices('temp_bott ~ temp_surf+depth_m+chla_surf+ph_surf', data=summer_mean, return_type='dataframe')

#create DataFrame to hold VIF values
vif_df = pd.DataFrame()
vif_df['variable'] = X.columns 

#calculate VIF for each predictor variable 
vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#view VIF for each predictor variable 
print(vif_df)


    variable          VIF
0  Intercept  7382.533872
1  temp_surf     1.932516
2    depth_m     1.753399
3  chla_surf     1.500366
4    ph_surf     1.375298


### Ordinary Least Squares Model

In [4]:
# predict_vars=["chla_surf","depth_m","temp_surf","ph_surf"]
X = summer_mean[["chla_surf","depth_m","temp_surf","ph_surf"]]
y = summer_mean["temp_bott"]

X_train=X[:42]
y_train=y[:42]
X_train=ssm.add_constant(X_train)        #to add constant value in the model

X_test=X[42:]
y_test=y[42:]
X_test=ssm.add_constant(X_test)        #to add constant value in the model

model= ssm.OLS(y_train,X_train).fit()         #fitting the model
predictions= model.summary()      #summary of the model
predictions

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_bott   R-squared:                       0.933
Model:                            OLS   Adj. R-squared:                  0.926
Method:                 Least Squares   F-statistic:                     128.9
Date:                Thu, 19 Sep 2024   Prob (F-statistic):           3.44e-21
Time:                        12:22:22   Log-Likelihood:                -16.753
No. Observations:                  42   AIC:                             43.51
Df Residuals:                      37   BIC:                             52.19
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         39.9652      5.727      6.979      0.000      28.362      51.569
chla_surf      0.1315      0.022      6.071      0.000       0.088       0.175
depth_m       -0.1124      0.010    -11.123      0.000      -0.133      -0.092
temp_surf      0.9614      0.141      6.831      0.000       0.676       1.247
ph_surf       -4.9844      0.868     -5.742      0.000      -6.743      -3.226
==============================================================================
Omnibus:                        6.016   Durbin-Watson:                   1.302
Prob(Omnibus):                  0.049   Jarque-Bera (JB):                8.043
Skew:                          -0.181   Prob(JB):                       0.0179
Kurtosis:                       5.113   Cond. No.                     3.28e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.28e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Spatial Regression

Apply Regressor across spatial raster datasets

In [12]:
# Fit model to FULL dataset
X = summer_mean[["depth_m","temp_surf"]]
y = summer_mean["temp_bott"]
X=ssm.add_constant(X)   
model= ssm.OLS(y,X).fit()         #fitting the model
predictions= model.summary()
predictions

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              temp_bott   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     116.6
Date:                Thu, 19 Sep 2024   Prob (F-statistic):           2.49e-23
Time:                        12:32:51   Log-Likelihood:                -43.879
No. Observations:                  57   AIC:                             95.76
Df Residuals:                      53   BIC:                             103.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             12.6945      4.123      3.079      0.003       4.424      20.965
depth_m           -0.1522      0.011    -13.912      0.000      -0.174      -0.130
temp_surf          0.5020      0.137      3.673      0.001       0.228       0.776
turbidity_surf     0.1908      0.062      3.093      0.003       0.067       0.315
==============================================================================
Omnibus:                       21.474   Durbin-Watson:                   1.703
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               29.804
Skew:                           1.416   Prob(JB):                     3.37e-07
Kurtosis:                       5.128   Cond. No.                     1.84e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [6]:

# model= ssm.OLS(y,X).fit() 

# Prepare raster datasets
sst_fp="../data/sea_surface_temperature/sst_summer_mean_2002_2020.tif"
depth_fp="../data/bathymetry/gebco_2024_n23.2842_s21.6651_w112.5659_e114.5956.tif"

sst_summer_mean=rio.open_rasterio(sst_fp).rio.write_crs(4326)
sst_summer_mean=sst_summer_mean
depth=xr.open_dataset(depth_fp).rio.write_crs(4326)
depth=depth.elevation * -1
depth=depth.where(depth > 0 ).rio.write_crs(4326)
# Match Rasters to depth raster
sst_summer_mean=sst_summer_mean.rio.reproject_match(depth)
sst_summer_mean=sst_summer_mean.where(sst_summer_mean > 0)


In [7]:
# Format raster data into arrays shaped [[v1,v2,v3...]] for model prediction
depth_vals=depth.values.flatten()
ss_vals=sst_summer_mean.values.flatten()

X_predict=[[float(depth_vals[x]),float(ss_vals[x])] for x in range(0,len(depth_vals))]

y_predict=np.array([float(model.predict([1.0,X_predict[x][0], X_predict[x][1]])[0]) for x in range(0,len(X_predict))]).reshape(depth.shape)
y_predict_ds=xr.DataArray(data=y_predict,coords=depth.coords).rio.write_crs(4326)

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error,max_error
import numpy as np

# Generate some sample data

# Calculate loss metrics based on EPD Data
bott_temp_validation=summer_mean.copy()
bott_temp_validation["pred_temp_bott"]=summer_mean[['latitude','longitude']].apply(lambda st: float(y_predict_ds.sel(x=st.longitude,y=st.latitude, method="nearest")), axis=1)
bott_temp_validation.dropna(subset=["pred_temp_bott"], inplace=True)

y_true=bott_temp_validation.temp_bott
y_pred=bott_temp_validation.pred_temp_bott

mae=mean_absolute_error(y_true,y_pred)
mse=mean_squared_error(y_true,y_pred)
rmse=root_mean_squared_error(y_true,y_pred)
mape=mean_absolute_percentage_error(y_true,y_pred)
me=max_error(y_true,y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Max Error: {me}")

Mean Absolute Error: 0.6330958350129284
Mean Squared Error: 0.9470119830329263
Root Mean Squared Error: 0.9731454069320403
Mean Absolute Percentage Error: 0.02478054812679395
Max Error: 4.990657935492745


In [10]:
# y_predict_ds.where(depth>0).rio.to_raster("../data/bottom_sst_predictions/predicted_bottom_temp.tif")
# depth.rio.to_raster("../data/bottom_sst_predictions/depth.tif")
# sst_summer_mean.where(depth>0).rio.to_raster("../data/bottom_sst_predictions/sst_summer_mean.tif")